In [1]:
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
import torch
from pathlib import Path
from rdkit import Chem
from lightning import pytorch as pl
from sklearn.metrics import mean_squared_error
import toolsets.torch_utilities as tu
from chemprop import data, featurizers, models, nn
from chemprop.data import MoleculeDatapoint, MoleculeDataset, build_dataloader
from chemprop.featurizers.molgraph.molecule import SimpleMoleculeMolGraphFeaturizer
from chemprop.nn.message_passing.base import BondMessagePassing, AtomMessagePassing

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:
data_train = pd.read_csv('mini_dataset/train_alc.csv')
data_test = pd.read_csv('mini_dataset/test_alc.csv')

In [3]:

num_workers = 4 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['rt'] # list of names of the columns containing targets

In [5]:
from chemprop.utils import make_mol
def transform_chemprop_df(df_input, smiles_column='smiles', target_columns=['rt']):
    smis = df_input.loc[:, smiles_column].values
    # mols = [make_mol(smi, keep_h=False, add_h=False) for smi in smis]
    ys = df_input.loc[:, target_columns].values
    all_data = [MoleculeDatapoint.from_smi(smi, y,
                                           x_d = tu.rdkit_featurzier(smi)
                                           ) for smi, y in zip(smis, ys)]
    return all_data
    # mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
    # train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.2, 0))
    # train_data, val_data, _ = data.split_data_by_indices(
    #     all_train, train_indices, val_indices, test_indices
    # )
    # featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
    # data_des = data.MoleculeDataset(all_data, featurizer)
    

    # featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
    # smis = data_train.loc[:, smiles_column].values
    # mols = [make_mol(smi, keep_h=False, add_h=False) for smi in smis]
    # ys = data_train.loc[:, target_columns].values       
    # cp_data = [MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]
    # # cp_data = data.MoleculeDataset(cp_data, featurizer)

In [6]:
all_train= transform_chemprop_df(data_train)
all_test = transform_chemprop_df(data_test)

[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerator
[19:12:15] DEPRECATION WARNING: please use MorganGenerat

In [7]:
mols = [d.mol for d in all_train]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.2, 0))
train_data, val_data, test_data = data.split_data_by_indices(
    all_train, train_indices, val_indices, test_indices
)
test_data = all_test

In [14]:
from chemprop.featurizers.atom import AtomFeatureMode

In [16]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer(atom_featurizer=AtomFeatureMode)

train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

test_dset = data.MoleculeDataset(test_data, featurizer)
val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

# test_dset = data.MoleculeDataset(test_data, featurizer)

StandardScaler()

In [18]:
featurizer.atom_fdim

3

In [9]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/chemprop/data/dataloader.py:52: UserWarning: Dropping last batch of size 1 to avoid issues with batch normalization (dataset size = 257, batch_size = 64)
  warnings.warn(


# lets keep working on that one tmr

In [10]:
# n_extra_atom_descriptors = 202
mp = nn.BondMessagePassing()
agg = nn.MeanAggregation()
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(input_dim = 502,output_transform=output_transform)# 300 hidden layers +202 descriptors
batch_norm = True
metric_list = [nn.metrics.RMSEMetric(), nn.metrics.MAEMetric()]# only the first one is used
# print(nn.metrics.MetricRegistry)# whoudl be somewhere in predictor layer
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=502, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSELoss(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
)

In [12]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=100, # number of epochs to train for
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(mpnn, train_loader, val_loader)

/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/fanzhoukong/Documents/GitHub/Rhoeto/checkpoints exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.
/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn              | BatchNorm1d        | 600    | train
3 | predictor       | RegressionFFN      | 151 K  | train
4 | X_d_transform   | Identity           | 0      | train
-----------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  1.78it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/chemprop/nn/message_passing/base.py:263: UserWarning: The operator 'aten::scatter_reduce.two_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  M_all = torch.zeros(len(bmg.V), H.shape[1], dtype=H.dtype, device=H.device).scatter_reduce_(


Epoch 6:   0%|          | 0/17 [00:00<?, ?it/s, train_loss=0.0742, val_loss=0.468]         

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x14737efc0>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1441, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/multiprocessing/conne

Epoch 6: 100%|██████████| 17/17 [00:15<00:00,  1.11it/s, train_loss=0.186, val_loss=0.468] 

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/opt/homebrew/Caskroom/miniforge/base/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/trainer/connector

Predicting DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 26.94it/s]


In [186]:
test_preds = np.concatenate(test_preds, axis=0)
# df_test['pred'] = test_preds
# df_test
data_test['pred'] = test_preds
mse = mean_squared_error(data_test['pred'], data_test['rt'])
print('the rmse on the test set is: ', np.sqrt(mse))
data_test['abs_offset']=abs(data_test['rt']-data_test['pred'])
data_test['abs_offset'].median()
print('the mean aboslute error is ', data_test['abs_offset'].median())  
from scipy.stats import pearsonr
pearsonr(data_test['pred'], data_test['rt'])
print('the pearson correlation is ', pearsonr(data_test['pred'], data_test['rt']))

the rmse on the test set is:  17.755846449957318
the mean aboslute error is  8.247017551094078
the pearson correlation is  PearsonRResult(statistic=0.973094953368522, pvalue=4.5485456595866265e-206)


7.663725559859408

PearsonRResult(statistic=0.9777189618079585, pvalue=5.185010897857018e-219)